In [1]:
# Author: Sunny You
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

2023-09-10 00:02:17.775758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
os.chdir("/Users/sunnyyou/Documents/Real_Time_HAI/HRICNN")

In [2]:
train = pd.read_csv("IMERG/Model_Data/ATL_train_resample.csv")
train = train[["GIS_ID", "DATE", "VMAX", "SHIPS_PER", "SHIPS_POT_Avg24h", "SHIPS_NOHC_Avg24h", "SHIPS_SHDC_Avg24h", "SHIPS_CFLX_Avg24h", "SHIPS_D200_Avg24h", "SHIPS_MTPW_108h", "SHIPS_PC2", "SHIPS_IR00_12h", "Category", "RI"]]
train.set_axis(["GIS_ID", "DATE", "VMAX", "PER", "POT", "NOHC", "SHDC", "ICDA", "D200", "TPW", "PC2", "SDBT", "Category", "RI"], axis = 1, inplace = True)
train

,GIS_ID,DATE,VMAX,PER,POT,NOHC,SHDC,ICDA,D200,TPW,PC2,SDBT,Category,RI
0,ATL_200004_TD_2000080900,2000080900,30,0,104.8,47.6,130.8,151.6,21.4,566,71.000000,139,TD,0
1,ATL_200004_TD_2000080906,2000080906,30,0,106.4,54.4,109.0,149.8,17.4,364,-1.666667,208,TD,0
2,ATL_200004_TD_2000080912,2000080912,30,0,107.4,59.0,115.6,137.8,21.4,422,-20.333333,176,TD,0
3,ATL_200004_TD_2000080918,2000080918,30,0,107.2,62.2,118.6,123.2,29.2,484,10.000000,141,TD,0
4,ATL_200004_TD_2000081000,2000081000,30,0,108.2,61.0,97.4,122.4,31.4,593,17.000000,151,TD,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7135,ATL_201709_TS_2017082406,2017082406,50,15,111.2,65.8,113.8,119.4,4.6,15,71.333333,146,TS,1
7136,ATL_201712_C3_2017090800,2017090800,105,15,43.6,57.8,141.4,590.0,51.4,197,139.500000,141,Maj,1
7137,ATL_200706_TS_2007090112,2007090112,50,15,94.0,34.4,93.6,248.2,16.0,212,-65.000000,121,TS,1
7138,ATL_200512_C3_2005082718,2005082718,100,5,44.0,89.2,59.0,332.4,14.8,0,-9.000000,50,Maj,1


In [3]:
test = pd.read_csv("IMERG/Model_Data/ATL_2018_test.csv")
test = test[["GIS_ID", "DATE", "VMAX", "SHIPS_PER", "SHIPS_POT_Avg24h", "SHIPS_NOHC_Avg24h", "SHIPS_SHDC_Avg24h", "SHIPS_CFLX_Avg24h", "SHIPS_D200_Avg24h", "SHIPS_MTPW_108h", "SHIPS_PC2", "SHIPS_IR00_12h", "Category", "RI"]]
test.set_axis(["GIS_ID", "DATE", "VMAX", "PER", "POT", "NOHC", "SHDC", "ICDA", "D200", "TPW", "PC2", "SDBT", "Category", "RI"], axis = 1, inplace = True)
test

,GIS_ID,DATE,VMAX,PER,POT,NOHC,SHDC,ICDA,D200,TPW,PC2,SDBT,Category,RI
0,ATL_201801_TS_2018052800,2018052800,55,15,60.2,2.2,158.0,189.4,62.0,192,-24.666667,160,TS,0
1,ATL_201801_TS_2018052806,2018052806,55,10,55.8,1.8,156.2,221.2,51.4,210,20.333333,136,TS,0
2,ATL_201801_TS_2018052812,2018052812,50,-5,48.4,1.4,161.2,207.8,42.0,46,6.666667,156,TS,0
3,ATL_201801_TS_2018052818,2018052818,45,-10,30.8,1.4,169.8,153.4,35.2,166,28.333333,205,TS,0
4,ATL_201802_TS_2018070500,2018070500,35,5,51.8,1.0,52.4,162.6,22.8,58,2.000000,178,TS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,ATL_201920_TS_2019112218,2019112218,55,5,38.2,0.8,394.0,394.8,47.0,961,-201.333333,327,TS,0
435,ATL_201920_TS_2019112300,2019112300,60,10,25.2,0.0,381.2,406.6,43.6,981,-211.000000,295,TS,0
436,ATL_201920_TS_2019112306,2019112306,60,5,17.0,0.0,376.8,379.0,45.4,978,-134.000000,159,TS,0
437,ATL_201920_TS_2019112312,2019112312,55,-5,14.6,0.0,372.8,324.0,44.0,962,-156.000000,187,TS,0


In [4]:
train_img = []
train_ships = []
train_label = []
test_img = []
test_ships = []
test_label = []
for f in range(len(train.GIS_ID)) :
    filename = "IMERG_Data_Old/IMERG_CSV/" + train.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[30:91, 30:91]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.RI[f]
        train_label.append(lab)
        ships = np.array([train.VMAX[f], train.PER[f], train.POT[f], train.NOHC[f], train.SHDC[f], train.ICDA[f], train.D200[f], train.TPW[f], train.PC2[f], train.SDBT[f]])
        train_ships.append(ships)
    except Exception as e:
        pass

for f in range(len(test.GIS_ID)) :
    filename = "IMERG_Data_Old/IMERG_CSV/" + test.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp.iloc[30:91, 30:91]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.RI[f]
        test_label.append(lab)
        ships = np.array([test.VMAX[f], test.PER[f], test.POT[f], test.NOHC[f], test.SHDC[f], test.ICDA[f], test.D200[f], test.TPW[f], test.PC2[f], test.SDBT[f]])
        test_ships.append(ships)
    except Exception as e:
        pass

In [5]:
print(len(train_img))
print(len(train_ships))
print(len(train_label))
print(len(test_img))
print(len(test_ships))
print(len(test_label))

7140
7140
7140
439
439
439


In [7]:
X_train_img = train_img
X_train_ships = train_ships
y_train = train_label
X_test_img = test_img
X_test_ships = test_ships
y_test = test_label

In [8]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,61,61,1)
X_train_img = X_train_img.astype('float32')
X_train_ships = np.array(X_train_ships)
X_train_ships = X_train_ships.reshape(-1,10)
y_train = np.array(y_train)

In [9]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,61,61,1)
X_test_img = X_test_img.astype('float32')
X_test_ships = np.array(X_test_ships)
X_test_ships = X_test_ships.reshape(-1,10)
y_test = np.array(y_test)

In [32]:
#300km
ships_input = keras.Input(shape =(10,), name = "ships_layer")
img_input = keras.Input(shape =(61, 61, 1), name = "img_layer")

w = keras.layers.Conv2D(64,12) (img_input)
w = keras.layers.Conv2D(64,12)(w)
w = keras.layers.Conv2D(64,2)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.linear(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(256,2)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, ships_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(1, activation = 'sigmoid')(output_layer1)

new_model1 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 12) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,2, activation = 'linear')(x)
x = keras.layers.Conv2D(128,7)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,2, activation = 'linear')(x)
x = keras.layers.Conv2D(64,4)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, ships_input])
output_layer2 = keras.layers.Dense(1, activation = 'sigmoid')(merged_model2)

new_model2 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (10,4))(img_input)
y = keras.layers.Conv2D(256, (4,10))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,6)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, ships_input])
output_layer3 = keras.layers.Dense(1, activation = 'sigmoid')(merged_model3)

new_model3 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 10)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, ships_input])
output_layer4 = keras.layers.Dense(1, activation = 'sigmoid')(merged_model4)

new_model4 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_layer (InputLayer)          [(None, 61, 61, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 50, 50, 64)   9280        img_layer[0][0]                  
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 39, 39, 64)   589888      conv2d_48[0][0]                  
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 38, 38, 64)   16448       conv2d_49[0][0]                  
____________________________________________________________________________________________

In [ ]:
new_model1.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits = False),
              metrics=['mae', 'mse'])
new_model2.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits = False),
              metrics=['mae', 'mse'])
new_model3.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits = False),
              metrics=['mae', 'mse'])
# new_model4.compile(optimizer='adam',
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits = False),
#               metrics=['mae', 'mse'])

In [19]:
new_model1.fit([X_train_img, X_train_ships], y_train, epochs=6, batch_size=1, validation_split = 0.1)
res = new_model1.evaluate([X_test_img, X_test_ships], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))
preds = new_model1.predict([X_test_img, X_test_ships])
a = np.average(preds, axis = 1)
test['preds'] = a

Epoch 1/8
6426/6426 [==============================] - 7562s 1s/step - loss: 1.7674 - mae: 0.3393 - mse: 0.2618 - val_loss: 0.5624 - val_mae: 0.3033 - val_mse: 0.1785
Epoch 2/8
6426/6426 [==============================] - 3819s 594ms/step - loss: 0.6459 - mae: 0.2943 - mse: 0.1924 - val_loss: 0.2314 - val_mae: 0.1496 - val_mse: 0.0691
Epoch 3/8
6426/6426 [==============================] - 3818s 594ms/step - loss: 0.5622 - mae: 0.2674 - mse: 0.1691 - val_loss: 1.1896 - val_mae: 0.4589 - val_mse: 0.3362
Epoch 4/8
6426/6426 [==============================] - 3811s 593ms/step - loss: 0.5023 - mae: 0.2431 - mse: 0.1515 - val_loss: 0.3007 - val_mae: 0.1945 - val_mse: 0.0911
Epoch 5/8
6426/6426 [==============================] - 3782s 589ms/step - loss: 0.4214 - mae: 0.2036 - mse: 0.1261 - val_loss: 0.4036 - val_mae: 0.2126 - val_mse: 0.1183
Epoch 6/8
2902/6426 [============>.................] - ETA: 33:43 - loss: 0.3023 - mae: 0.1549 - mse: 0.0897

KeyboardInterrupt: 

In [ ]:
test